In [1]:
import numpy as np
import scipy.stats
import scs

In [2]:
#############################################
#      Generate random cone problems        #
#############################################

def pos(x):
    return (x + np.abs(x)) / 2.
    
def gen_feasible(m, n, p_scale = 0.1):
    z = np.random.randn(m)
    y = pos(z)
    s = y - z

    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    # Make problem slightly more numerically challenging:
    A = np.random.randn(m, n)
    U, S, V = np.linalg.svd(A, full_matrices=False)
    S = S**2
    S /= np.max(S)
    A = (U * S) @ V
        
    x = np.random.randn(n)
    c = -A.T @ y - P @ x
    b = A.dot(x) + s
    
    #b /= np.linalg.norm(b)
    #x /= np.linalg.norm(b)
    
    #c /= np.linalg.norm(c)
    #y /= np.linalg.norm(c)
    
    return (P, A, b, c, x, y)

In [3]:
def gen_infeasible(m, n, p_scale = 0.1):
    # b'y < 0, A'y == 0
    z = np.random.randn(m)
    y = pos(z)  # y = s - z;
    
    A = np.random.randn(m, n)
    
    # A := A - y(A'y)' / y'y
    A = A - np.outer(y, np.transpose(A).dot(y)) / np.linalg.norm(y)**2

    b = np.random.randn(m)
    b = -b / np.dot(b, y)
    
    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    return (P, A, b, np.random.randn(n))

def gen_unbounded(m, n, p_scale = 0.1):
    # c'x < 0, Ax + s = 0, Px = 0
    z = np.random.randn(m)
    s = pos(z)
    
    P = np.random.randn(n,n)
    P = p_scale * P.T @ P
    eigs, V = np.linalg.eig(P)
    
    i = np.argmin(eigs)
    eigs[i] = 0
    x = V[:,i]
    
    # Px = 0
    P = (V * eigs) @ V.T
    P = 0.5 * (P + P.T)
      
    A = np.random.randn(m, n)

    # A := A - (s + Ax)x' / x'x ===> Ax + s == 0
    A = A - np.outer(s + A.dot(x), x) / np.linalg.norm(x)**2
        
    c = np.random.randn(n)
    c = -c / np.dot(c, x)
    
    return (P, A, np.random.randn(m), c)

In [4]:
def is_optimal(P, A, b, c, x, y, tol=1e-6):
    s = b - A @ x
    
    if (np.linalg.norm(P @ x + c + A.T @ y) < tol and
        np.abs(y.T @ s) < tol and
        np.linalg.norm(s - pos(s)) < tol and
        np.linalg.norm(y - pos(y)) < tol):
        return True
    return False

def is_infeasible(A, b, y, tol=1e-6):
    if b.T @ y >= 0:
        return False
    
    y_hat = y / np.abs(b.T @ y)
    if (np.linalg.norm(y_hat - pos(y_hat)) < tol and np.linalg.norm(A.T @ y_hat) < tol):
        return True
    return False

def is_unbounded(P, A, c, x, tol=1e-6):
    if c.T @ x >= 0:
        return False
    
    x_hat = x / np.abs(c.T @ x)
    if np.linalg.norm(P @ x_hat) < tol and np.linalg.norm(A @ x_hat + pos(-A @ x_hat)) < tol:
        return True
    return False

In [5]:
# ''linear'' projection logic

class LinearProjector(object):
    def __init__(self, P, A, b, c):
        (m,n) = A.shape
        self.A = A
        self.h = np.hstack((c, b))        
        self.L = scipy.linalg.cho_factor(np.eye(n) + P + A.T @ A)
        self.g = self._solve(self.h)

    def _solve(self, v):
        (m, n) = self.A.shape
        sol = np.zeros(n+m,)
        sol[:n] = scipy.linalg.cho_solve(self.L, v[:n] - self.A.T @ v[n:])
        sol[n:] = v[n:] + self.A @ sol[:n]
        return sol
        
    def project(self, w):
        g = self.g
        p = self._solve(w[:-1])
        
        _a = 1 + g.T@g
        _b = w[:-1].T @ g - 2 * p.T @ g - w[-1]
        _c = p.T@p - w[:-1].T @ p
        
        tau = (-_b + np.sqrt(_b ** 2 - 4 * _a * _c)) / 2 / _a
        
        return np.hstack((p - tau * g, tau))

# Douglas-Rachford splitting / ADMM for QP
def solve_qp_dr(P, A, b, c, N=1000, tol=1e-6):
    (m,n) = np.shape(A)

    lp = LinearProjector(P, A, b, c)

    u = np.zeros(n+m+1,)
    #u[:n+m] = np.hstack((c, b))
    u[-1] = 1.

    def proj_cone(v):
        v[n:] = np.maximum(v[n:], 0.)
        return v


    use_dr = True # slightly different DR vs ADMM
    
    for i in range(N):
        # for DR ut converges to sol *not* u, see Patrinos, Stella, Bemporad, 2014
        # v - ut go to zero
        dr_lam = 0.5 # \in [0,1], 0.5 = DR, 1.0 = PR

        ut = lp.project(u)
        v = proj_cone(2 * ut - u)
        u += 2 * dr_lam * (v - ut)
        
        x = ut[:n] / ut[-1]
        y = ut[n:-1] / ut[-1]
        
        if (is_optimal(P, A, b, c, x, y, tol=tol) or 
            is_infeasible(A, b, y, tol=tol) or
            is_unbounded(P, A, c, x, tol=tol)):
            break
        
    print(i)
    return x,y


In [6]:
m = 1500
n = 1000
N = int(5e3)
seed = 1234

In [7]:
np.random.seed(seed)
(P, A, b, c) = gen_unbounded(m, n)
#P = P + 1e-7 * np.eye(n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=10 * N, acceleration_lookback = 10, eps=1e-6)

4825
------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-03, alpha: 1.50, max_iters: 50000,
	  normalize: 1, scale: 1.00, rho_x: 1.00e-03,
	  acceleration_lookback: 10
lin-sys:  sparse-direct
	  nnz(A): 1500000, nnz(P): 500500
------------------------------------------------------------------
 Iter | pri res | dua res | rel gap |   obj   | kap/tau | time (s)
------------------------------------------------------------------
     0| 1.37e+18  1.66e+18  2.62e-01 -5.13e+19  1.53e+20  2.78e-02 
   100| 1.04e+13  1.32e+13  1.00e+00 -3.98e+18  3.34e+11  9.39e-01 
   120| 3.60e+11  5.06e+11  1.00e+00 -3.98e+18  3.34e+10  1.12e+00 
------------------------------------------------------------------
status:  unbounded
timings: to

In [8]:
print(c.T @ sol['x'])
print(np.linalg.norm(c) * np.linalg.norm(A @ sol['x'] + sol['s']))
print(np.sqrt(max(0., sol['x'].T @ P @ sol['x'])))

-1.0000000000000002
0.0005156027452573967
2.6633655930009557e-06


In [9]:
print(np.linalg.norm(P - P.T))

0.0


In [ ]:
np.random.seed(seed)
(P, A, b, c) = gen_infeasible(m, n)
#P = P + np.eye(n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

/usr/local/google/home/bodonoghue/miniconda2/envs/scs_qp/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/google/home/bodonoghue/miniconda2/envs/scs_qp/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/google/home/bodonoghue/miniconda2/envs/scs_qp/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in matmul
  
/usr/local/google/home/bodonoghue/miniconda2/envs/scs_qp/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in matmul
  after removing the cwd from sys.path.
/usr/local/google/home/bodonoghue/miniconda2/envs/scs_qp/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in matmul
  if sys.path[0] == '':
/usr/local/google/home/bodonoghue/miniconda2/envs/scs_qp/lib/python3.6/site-packages/ipykernel_launcher.py:1

519
------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, rho_x: 1.00e-03,
	  acceleration_lookback: 10
lin-sys:  sparse-direct
	  nnz(A): 1500000, nnz(P): 500500
------------------------------------------------------------------
 Iter | pri res | dua res | rel gap |   obj   | kap/tau | time (s)
------------------------------------------------------------------
     0| 1.21e+03  7.83e+02  9.94e-01  8.61e+05  0.00e+00  2.79e-02 


In [ ]:
print(b.T @ sol['y'])
print(np.linalg.norm(b) * np.linalg.norm(A.T @ sol['y']))

In [ ]:
np.random.seed(seed)
(P, A, b, c, _x, _y) = gen_feasible(m, n, p_scale=0.1)
#(_x,_y) = solve_qp_dr(P, A, b, c, N=int())
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)

cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-9, scale=0.1)

In [ ]:
x = sol['x']
y = sol['y']
s = sol['s']

print(_x.T@ P @ _x / 2 + c.T @ _x)
print(x.T@ P @ x / 2 + c.T @ x)

print(np.linalg.norm(x - _x))
print(np.linalg.norm(y - _y))

#x = _x
#y = _y
#s = b - A @ _x

print(np.linalg.norm(A @ x + s - b) / (1+np.linalg.norm(b)))
print(np.linalg.norm(P@x + A.T @ y + c) / (1+np.linalg.norm(c)))
print(abs(x.T@ P @ x + c.T @ x + b.T @ y) / (1 + abs(x.T@P @ x) + abs(c.T @ x) + abs(b.T @ y)))

In [ ]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
#(x,y) = solve_qp_dr(P, A, b, c, N=N)
#print(c.T @ x)
#print(x)

In [ ]:
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=False)
print(c.T @ sol['x'])

In [ ]:
probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

In [ ]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, 0.001)

In [ ]:
print(np.linalg.norm(A.flatten(), np.inf))
print(np.linalg.norm(P.flatten(), np.inf))

In [ ]:
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)
print(y)

In [ ]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)

cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-6, scale=1)

In [ ]:
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 5, eps=1e-6, use_indirect=True, scale=1)

In [ ]:
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=True, scale=1)

In [ ]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

In [ ]:
sol = scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 0, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

In [ ]:
probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])

In [ ]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
print(np.linalg.norm(A))

In [ ]:
np.random.seed(seed)
(P, A, b, c, _, _) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(c.T @ x)
print(x)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)

probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6, use_indirect=True)
print(c.T @ sol['x'])